In [ ]:
import fastf1
from matplotlib import pyplot as plt
import fastf1
import fastf1.plotting as f1_plt
import pandas as pd

### session

In [ ]:
session = fastf1.get_session(2022, 21, "Q")
session.load(telemetry=True, laps=True, weather=True)

In [ ]:
session = fastf1.get_session(2022, 21, 'Q')
session.load(laps=True, telemetry=True, weather=True)
laps = session.laps

q1, q2, q3 = laps.split_qualifying_sessions()
wheater = q3.get_weather_data()

fast_mag = q3.pick_driver('MAG')
mag_car_data = fast_mag.get_car_data()
t_mag = mag_car_data['SessionTime']
vCar_mag = mag_car_data['Speed']

fast_ver = q3.pick_driver('VER')
ver_car_data = fast_ver.get_car_data()
t_ver = ver_car_data['SessionTime']
vCar_ver = ver_car_data['Speed']
mag_car_data

In [ ]:
t_mag = mag_car_data['Date']
t_ver = ver_car_data['Date']


# The rest is just plotting
fig, ax = plt.subplots()
ax.plot(t_mag, vCar_mag, label='Mag')
ax.plot(t_ver, vCar_ver, label='Ver')
# ax.plot(wheater['Time'], wheater['WindDirection'], label='Wheater')
ax.set_ylabel('Speed [Km/h]')
ax.set_title('MAG is')
ax.legend()
plt.show()

In [ ]:
session = fastf1.get_session(2022, 21, 'Q')
session.load(laps=True, telemetry=True, weather=True)
laps = session.laps

q1, q2, q3 = laps.split_qualifying_sessions()
wheater = q3.get_weather_data()

fast_mag = q3.pick_driver('MAG')
mag_car_data = fast_mag.get_car_data()
t_mag = mag_car_data['SessionTime']
vCar_mag = mag_car_data['Speed']

fast_ver = q3.pick_driver('VER')
ver_car_data = fast_ver.get_car_data()
t_ver = ver_car_data['SessionTime']
vCar_ver = ver_car_data['Speed']

### Best lap and showing track

In [ ]:
s1_mag = fast_mag.pick_fastest()["Sector1Time"] 
s1_ver = fast_ver.pick_fastest()["Sector1Time"] 
 
s2_mag = fast_mag.pick_fastest()["Sector2Time"] 
s2_ver = fast_ver.pick_fastest()["Sector2Time"] 

s3_mag = fast_mag.pick_fastest()["Sector3Time"] 
s3_ver = fast_ver.pick_fastest()["Sector3Time"] 

s1 = fast_mag.pick_fastest()["Sector1Time"] < fast_ver.pick_fastest()["Sector1Time"] 
s2 = fast_mag.pick_fastest()["Sector2Time"] < fast_ver.pick_fastest()["Sector2Time"] 
s3 = fast_mag.pick_fastest()["Sector3Time"] < fast_ver.pick_fastest()["Sector3Time"] 
print(s1_mag, s1_ver)
print(s2_mag, s2_ver)
print(s3_mag, s3_ver)

print(s1,s2,s3)

In [ ]:
best_lap_mag = fast_mag.pick_lap(25).get_car_data()
best_lap_tel_mag = fast_mag.pick_lap(25).get_telemetry()

best_lap_ver = fast_ver.pick_lap(22).get_car_data()
best_lap_tel_ver = fast_ver.pick_lap(22).get_telemetry()

x_mag = best_lap_tel_mag["X"]
y_mag = best_lap_tel_mag["Y"]
x_ver = best_lap_tel_ver["X"]
y_ver = best_lap_tel_ver["Y"]

img = plt.imread("Brazil_Circuit.png")
fig, ax = plt.subplots()
ax.imshow(img, extent=[-6250, 6850, -2400, 5150])
ax.set_facecolor("white")
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

ax.plot(y_mag, -x_mag, label='MAG', c="white", linewidth=1)
plt.show()

### Difference in time

In [ ]:
best_lap_tel_mag["Distance"]

In [ ]:
idx = best_lap_tel_ver.index
best_lap_tel_ver["Time"]
best_lap_tel_mag["X"]
best_lap_tel_mag["Y"]


In [ ]:
fig, ax = plt.subplots()
for idx in best_lap_tel_mag.index:
    ver = best_lap_tel_ver["Time"][idx]
    mag = best_lap_tel_mag["Time"][idx]
    
    ahead = ver > mag
    ax.scatter(x_mag[idx], y_mag[idx], c='grey' if ahead else 'blue')
    print(ver, mag, ahead)  
plt.show()  

### Get differet sectors and mini sectors

In [ ]:
def IsTurn(x,y,x_last_turn,y_last_turn, turn_difference=1300):
    if x + turn_difference < x_last_turn:
        return True        
    elif x - turn_difference > x_last_turn:
        return True        
    elif y + turn_difference < y_last_turn:
        return True        
    elif y - turn_difference > y_last_turn:
        return True        
    else:
        return False

In [ ]:
def find_minisectors(lap_tel, turn_difference=1300):
    x_last_turn, y_last_turn = 0,0
    turn_count = 0
    turn_idx = []
    for idx, xy in enumerate(zip(lap_tel["X"], lap_tel["Y"]), start=2):
        x, y = xy
        if IsTurn(x, y, x_last_turn, y_last_turn, turn_difference):
            x_last_turn = x
            y_last_turn = y
            turn_count += 1
            turn_idx.append(idx)
    return turn_idx, turn_count

In [ ]:
idx, turn_count = find_minisectors(best_lap_tel_mag, turn_difference=800)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_mag,y_mag)
print(turn_count)
for i in idx:
    ax.scatter(x_mag[i], y_mag[i], c='red')

In [ ]:
def find_minisectors_times(lap_tel, turn_difference=800):
    mini_sector_times = []
    ## find idx of mini sectors
    idx, _ = find_minisectors(lap_tel, turn_difference)
    
    time = lap_tel["Time"]
    last_time = time[idx[0]]
    for i in idx[1:]:
        current_time = time[i]
        mini_sector_time = current_time - last_time
        mini_sector_times.append(mini_sector_time)
        last_time = time[i]
    return mini_sector_times

def find_minisectors_for_two(lap_tel_1, lap_tel_2, turn_difference=800):
    mini_sector_times_1 = []
    mini_sector_times_2 = []
    
    idx, _ = find_minisectors(lap_tel_1, turn_difference)
    
    time_1 = lap_tel_1["Time"]
    time_2 = lap_tel_2["Time"]
    prev_time_1 = time_1[idx[0]]
    prev_time_2 = time_2[idx[0]]

    for i in idx[1:]:
        current_time_1 = time_1[i]
        current_time_2 = time_1[i]

        mini_sector_time_1 = current_time_1 - prev_time_1
        mini_sector_time_2 = current_time_2 - prev_time_2

        mini_sector_times_1.append(mini_sector_time_1)
        mini_sector_times_2.append(mini_sector_time_2)
        
        prev_time_1 = time_1[i]
        prev_time_2 = time_2[i]
    return mini_sector_times_1, mini_sector_times_2

In [ ]:
def compare_minisectors(lap_tel_1, lap_tel_2, turn_difference=800):
    
    first_fastest = []

    mini_sectors_1, mini_sectors_2 = find_minisectors_for_two(lap_tel_1, lap_tel_2, turn_difference)

    for i in range(len(mini_sectors_1)):
        first_fastest.append(mini_sectors_1[i] < mini_sectors_2[i])
    
    return idx, first_fastest


In [ ]:
idx, first_fastest = compare_minisectors(best_lap_tel_mag, best_lap_tel_ver, turn_difference=800)

In [ ]:
magColor = fastf1.plotting.driver_color("mag")
verColor = fastf1.plotting.driver_color("ver")

In [ ]:
img = plt.imread("Brazil_Circuit.png")
fig, ax = plt.subplots()
ax.imshow(img, extent=[-6250, 6850, -2400, 5150])
ax.set_facecolor("white")
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

# ax.plot(x_mag,y_mag, color='black')
for index in x_mag.index:
    if index in idx[:]:
        color = magColor if first_fastest[idx.index(index) - 1] else verColor

    ax.scatter(y_mag[index], -x_mag[index], c=color, s=1)

plt.show()

### weather data

In [ ]:
df_weather = pd.read_csv('q3_wheater.csv', index_col=False)

In [ ]:
from datetime import datetime as dt
from datetime import timedelta
df_weather.sort_values("Time", inplace=True)

In [ ]:
def convert_string_to_time(string):
    string = string.split(" ")[-1]
    return dt.strptime(string, '%H:%M:%S.%f')

In [ ]:
def convert_delta_to_time(delta):
    datetime_0 = dt(1900,1,1,0,0,0)

    return (datetime_0 + delta)

In [ ]:
best_lap_mag_date_time = best_lap_mag["SessionTime"].apply(convert_delta_to_time)
best_lap_ver_date_time = best_lap_ver["SessionTime"].apply(convert_delta_to_time)

In [ ]:
## track temp over time
fig, ax = plt.subplots()
ax.xaxis.axis_date()
for idx, _  in enumerate(df_weather.index):

    isRainfall = df_weather.iloc[idx]["Rainfall"]
    timeString = df_weather.iloc[idx]["Time"]
    trackTemp = df_weather.iloc[idx]["TrackTemp"]
    timeStamp = convert_string_to_time(timeString)
    ax.plot(timeStamp, trackTemp, c="blue" if isRainfall else "black",  marker="o", markersize=5)
ax.plot(best_lap_mag_date_time, [26]*len(best_lap_mag["SessionTime"]), c="black")
ax.plot(best_lap_ver_date_time, [25]*len(best_lap_ver["SessionTime"]), c="blue")